## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tasiilaq,GL,65.6145,-37.6368,19.13,78,100,40.67,overcast clouds
1,1,Bilma,NE,18.6853,12.9164,57.99,22,3,6.67,clear sky
2,2,Punta Arenas,CL,-53.1500,-70.9167,57.22,62,0,8.05,clear sky
3,3,Woodward,US,36.4337,-99.3904,36.63,51,0,10.36,clear sky
4,4,Praia Da Vitoria,PT,38.7333,-27.0667,57.76,94,40,11.79,scattered clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for travel?"))
max_temp = float(input("What is the maximum temperature you would like for travel?"))


What is the minimum temperature you would like for travel?50
What is the maximum temperature you would like for travel?75


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
new_city_data_df = city_data_df.loc[(city_data_df["Max Temp"]>= min_temp) & (city_data_df["Max Temp"]< max_temp)]
new_city_data_df 

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Bilma,NE,18.6853,12.9164,57.99,22,3,6.67,clear sky
2,2,Punta Arenas,CL,-53.1500,-70.9167,57.22,62,0,8.05,clear sky
4,4,Praia Da Vitoria,PT,38.7333,-27.0667,57.76,94,40,11.79,scattered clouds
6,6,Rawson,AR,-43.3002,-65.1023,60.49,67,78,23.33,broken clouds
8,8,Esperance,AU,-33.8667,121.9000,62.44,70,2,15.03,clear sky
...,...,...,...,...,...,...,...,...,...,...
722,722,Zhuhai,CN,22.2769,113.5678,57.97,76,40,17.27,scattered clouds
727,727,Teguise,ES,29.0605,-13.5640,64.62,64,0,9.22,clear sky
729,729,Waitati,NZ,-45.7500,170.5667,57.79,57,100,5.68,overcast clouds
730,730,Camalu,MX,30.8500,-116.0667,67.42,41,2,6.98,clear sky


In [15]:
# 4a. Determine if there are any empty rows.
new_city_data_df.isna().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = new_city_data_df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 246 entries, 1 to 738
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              246 non-null    int64  
 1   City                 246 non-null    object 
 2   Country              246 non-null    object 
 3   Lat                  246 non-null    float64
 4   Lng                  246 non-null    float64
 5   Max Temp             246 non-null    float64
 6   Humidity             246 non-null    int64  
 7   Cloudiness           246 non-null    int64  
 8   Wind Speed           246 non-null    float64
 9   Current Description  246 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 21.1+ KB


In [30]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.info()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bilma,NE,57.99,clear sky,18.6853,12.9164,
2,Punta Arenas,CL,57.22,clear sky,-53.1500,-70.9167,
4,Praia Da Vitoria,PT,57.76,scattered clouds,38.7333,-27.0667,
6,Rawson,AR,60.49,broken clouds,-43.3002,-65.1023,
8,Esperance,AU,62.44,clear sky,-33.8667,121.9000,
9,Port Elizabeth,ZA,66.49,broken clouds,-33.9180,25.5701,
14,Lebu,CL,59.85,clear sky,-37.6167,-73.6500,
15,Ushuaia,AR,53.26,clear sky,-54.8000,-68.3000,
21,New Norfolk,AU,67.75,scattered clouds,-42.7826,147.0587,
23,Taft,IR,60.75,broken clouds,31.7471,54.2088,


In [32]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params ["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels["results"][0]["name"]
    except:
         print("Hotel not found ... skipping")
        
hotel_df

Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bilma,NE,57.99,clear sky,18.6853,12.9164,Expeditions Ténére voyages
2,Punta Arenas,CL,57.22,clear sky,-53.1500,-70.9167,Hotel Hain
4,Praia Da Vitoria,PT,57.76,scattered clouds,38.7333,-27.0667,Hotel Branco I
6,Rawson,AR,60.49,broken clouds,-43.3002,-65.1023,Hosteria Sampedro
8,Esperance,AU,62.44,clear sky,-33.8667,121.9000,Hospitality Esperance
...,...,...,...,...,...,...,...
722,Zhuhai,CN,57.97,scattered clouds,22.2769,113.5678,Nanyang Seascape Hotel Zhuhai Hotel
727,Teguise,ES,64.62,clear sky,29.0605,-13.5640,B&B La Casona de Nazaret
729,Waitati,NZ,57.79,overcast clouds,-45.7500,170.5667,Arden Country House BnB Dunedin
730,Camalu,MX,67.42,clear sky,30.8500,-116.0667,La Cueva del Pirata


In [39]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 246 entries, 1 to 738
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 246 non-null    object 
 1   Country              246 non-null    object 
 2   Max Temp             246 non-null    float64
 3   Current Description  246 non-null    object 
 4   Lat                  246 non-null    float64
 5   Lng                  246 non-null    float64
 6   Hotel Name           246 non-null    object 
dtypes: float64(3), object(4)
memory usage: 15.4+ KB


In [40]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [49]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


In [50]:
# 11a. Add a marker layer for each city to the map.
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))